## 1. Import Dependencies

In [24]:
# Import helpers
import os 
import numpy as np # numpy is a library that provides a multidimensional array object
import random  # random offers a suite of functions for generating random numbers

# Import gym stuff
import gym 
from gym import Env # Look here what the Env class can do: github.com/openai/gym/blob/master/gym/core.py
from gym.spaces import Tuple, Box, MultiBinary, MultiDiscrete, Discrete, Dict

# Import stable_baseline3 stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## 2. Build Environment
- Simple Idea: Building an Agent that gives us the perfect temeprature to cook a boiled egg in 10 minutes
- Perfect temperature should be between 70° and 80°

In [73]:
class CookingEnv(Env):
    
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 75 + random.randint(-3,3) # unit is degrees
        self.boil_lenght = 10 # unit is minute
    
    def reset(self, action):
        self.state = action-1
       
    
    def step(self, action):
        # Apply action
        self.state += action -1 
          #Explanation: 
           # 0 -1 = -1 temperature
           # 1 -1 = 0 
           # 2 -1 = 1 temperature 
        
     
        # Decreasing boil_lenght by 1 minute
        self.boil_lenght -= 1
        
        # Calculating reward
        if self.state <= 70 and self.state >= 80:
            reward = 1
        else:
            reward = -1
        
        if self.boil_lenght <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return info, self.state, done, reward
    
    def render(self):
        # You could implement viz here (More information: https://pypi.org/project/graphviz/)
        pass
    
    def reset(self):
        self.state = np.array([75 + random.randint(-3,3)]).astype(float)
        self.boil_lenght = 10
        return self.state

## 3. Test Environment

In [74]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        # env.render()
        action = env.action_space.sample()
        info, reward, done, obs = env.step(action)
        score += reward
    print(f'Episode: {episode}, Score: {score}')

Episode: 1, Score: -1
Episode: 2, Score: 3
Episode: 3, Score: -1
Episode: 4, Score: -2
Episode: 5, Score: 3


## 4. Train Model 

In [75]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log =log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [76]:
model.learn(total_timesteps = 1000)

Logging to Training\Logs\PPO_7


TypeError: float() argument must be a string or a number, not 'dict'

## 5. Save Model

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'CustomEnv')